In [ ]:
# Qwen3 智能对比基准测试

这个笔记本专门用于对比不同规模的 Qwen3 模型（8B vs 14B vs 32B）的智能表现差异。

## 🎯 测试目标
通过精心设计的中等难度问题，量化对比不同参数规模模型的智能差异，包括：
- 逻辑推理能力
- 数学计算能力  
- 知识理解深度
- 创意生成能力
- 代码实现质量
- 伦理思考深度
- 抽象思维能力
- 复杂推理能力

## 📋 快速开始

### 第一步：环境准备
运行下面的单元格，安装并启动 Ollama 服务。

### 第二步：下载模型
运行模型下载单元格，获取所需的 Qwen3 模型。

### 第三步：运行测试
执行实验函数，开始智能对比测试。

## ⚠️ 注意事项
- 支持模型：qwen3:8b, qwen3:14b, qwen3:32b
- 函数会自动识别模型规模并调整参数
- 32B模型需要更多内存和推理时间
- 测试包含8个维度的综合评估

In [ ]:
# === 1. 在 Colab 安装并启动 Ollama ===
import subprocess
import time
import os

print("⬇️ 正在安装 Ollama...")
install_cmd = "curl -fsSL https://ollama.com/install.sh | sh"
install_result = subprocess.run(install_cmd, shell=True, check=False)
if install_result.returncode == 0:
    print("✅ Ollama 安装完成")
else:
    print("⚠️ Ollama 安装遇到问题，继续尝试...")
    # 尝试备用安装方式
    try:
        subprocess.run(["apt", "update"], check=False)
        subprocess.run(["apt", "install", "-y", "curl"], check=False)
        install_result = subprocess.run(install_cmd, shell=True, check=False)
        if install_result.returncode == 0:
            print("✅ Ollama 安装完成（重试）")
        else:
            print("⚠️ Ollama 可能已安装或安装失败，继续执行...")
    except Exception as e:
        print(f"⚠️ 备用安装方式失败: {str(e)}，继续执行...")

print("🚀 正在后台启动 Ollama 服务...")
# 将进程句柄暴露在全局作用域，方便后续手动停止
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 服务已在 localhost:11434 运行，变量 ollama_server 可用于手动终止")

In [ ]:
# === 2. 下载所需模型（可根据需要增减） ===
import subprocess


print("⬇️ 正在下载 qwen3:8b...")
pull_qwen_8b = subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
if pull_qwen_8b.returncode != 0:
    raise RuntimeError("❌ qwen3:8b 下载失败")

print("⬇️ 正在下载 qwen3:14b...")
pull_qwen_14b = subprocess.run(["ollama", "pull", "qwen3:14b"], check=False)
if pull_qwen_14b.returncode != 0:
    raise RuntimeError("❌ qwen3:14b 下载失败")


#TODO 用l4的时候解除注释
# print("⬇️ 正在下载 qwen3:32b...")
# pull_qwen_32b = subprocess.run(["ollama", "pull", "qwen3:32b"], check=False)
# if pull_qwen_32b.returncode != 0:
#     raise RuntimeError("❌ qwen3:32b 下载失败")

print("✅ 模型下载完成，可以运行后续实验")

⬇️ 正在下载 qwen3:8b...
⬇️ 正在下载 qwen3:14b...
✅ 模型下载完成，可以运行后续实验


In [ ]:
# === 3. 导入必要的库和配置 ===
import requests
import json
import time

# 配置 Ollama API 地址
OLLAMA_URL = "http://localhost:11434"

print("✅ 依赖库导入完成")
print(f"🔧 Ollama API 地址: {OLLAMA_URL}")

In [ ]:
# 定义核心函数：Ollama API 调用函数（真正的流式显示 - 无降级逻辑）

def ollama_chat_qwen(messages, temperature=0.7, max_retries=3, model="qwen3:8b", 
                     debug=False, enable_thinking=False, show_thinking=False):
    """
    调用 Qwen3 模型进行推理
    
    参数:
        messages: 对话消息列表
        temperature: 温度参数 (0.0-1.0)
        max_retries: 最大重试次数
        model: 模型名称
        debug: 是否显示调试信息
        enable_thinking: 是否启用思考模式（默认False）
        show_thinking: 是否显示思考内容（仅当enable_thinking=True时有效）
    
    注意: 
        - 本地推理，无timeout限制
        - num_ctx=8192, num_predict=4096 (平衡显存与性能)
        - 真正的流式显示，实时打印每个token
        - 无降级逻辑，流式失败就失败
    """
    url = f"{OLLAMA_URL}/api/chat"
    
    # 优化后的参数配置
    options = {
        "temperature": temperature,
        "top_p": 0.9,
        "repeat_penalty": 1.07,  # 降低惩罚率，避免过度回避重复
        "num_ctx": 8192,  # 上下文窗口，T4/L4显存充足
        "num_predict": 4096  # 最大输出长度，防止生成过长卡死
    }
    
    # 构建请求payload
    payload = {
        "model": model,
        "messages": messages,
        "stream": True,  # 始终使用流式
        "think": enable_thinking,
        "options": options
    }
    
    # 模型显示名称
    model_size = model.split(":")[-1] if ":" in model else "8b"
    model_display_name = {
        "32b": "Qwen3 32B",
        "14b": "Qwen3 14B", 
        "8b": "Qwen3 8B",
        "1.7b": "Qwen3 1.7B"
    }.get(model_size, f"Qwen3 {model_size}")
    
    for attempt in range(max_retries):
        try:
            print(f"🤖 {model_display_name} 推理中... (第{attempt+1}次尝试)")
            if debug:
                print(f"   [调试] 本地推理：num_ctx=8192, num_predict=4096, repeat_penalty=1.07")
                print(f"   [调试] 本地推理：无timeout、无num_ctx、无num_predict限制")
            
            start_time = time.time()
            
            # 流式API处理
            response = requests.post(url, json=payload, stream=True)
            response.raise_for_status()
            
            result = ""
            thinking_text = ""
            
            print("💬 回答: ", end='', flush=True)  # 开始流式输出
            
            for line in response.iter_lines():
                if line:
                    try:
                        data = json.loads(line.decode('utf-8'))
                        
                        if 'message' in data:
                            # 提取 content 并实时打印
                            if 'content' in data['message']:
                                content = data['message']['content']
                                if content:
                                    print(content, end='', flush=True)  # 🌊 实时打印！
                                    result += content
                            
                            # 提取 thinking（如果存在）
                            if 'thinking' in data['message'] and data['message']['thinking']:
                                thinking_content = data['message']['thinking']
                                thinking_text += thinking_content
                                if show_thinking:
                                    print(thinking_content, end='', flush=True)
                        
                        if data.get('done', False):
                            print()  # 换行
                            break
                            
                    except json.JSONDecodeError:
                        continue
            
            end_time = time.time()
            
            # 检查结果
            if not result or result.strip() == "":
                if thinking_text and thinking_text.strip():
                    print(f"💭 content为空，使用thinking内容（{len(thinking_text)}字符）")
                    result = thinking_text
                else:
                    print(f"⚠️ {model_display_name} 返回内容为空")
                    if attempt < max_retries - 1:
                        print(f"   等待3秒后重试...")
                        time.sleep(3)
                        continue
                    else:
                        return f"[错误: 模型返回空内容，已重试{max_retries}次]"
            
            # 显示思考内容（如果需要）
            if show_thinking and thinking_text and thinking_text.strip():
                print(f"\n💭 思考过程（{len(thinking_text)}字符）:")
                print(f"{thinking_text[:300]}..." if len(thinking_text) > 300 else thinking_text)
            
            print(f"✅ 推理完成！耗时: {end_time - start_time:.1f}秒, 长度: {len(result)}字符")
            return result.strip()
            
        except requests.exceptions.RequestException as e:
            print(f"❌ 网络错误: {str(e)}")
            if attempt < max_retries - 1:
                print(f"   等待3秒后重试...")
                time.sleep(3)
            continue
            
        except Exception as e:
            print(f"❌ 未知错误: {str(e)}")
            if debug:
                import traceback
                traceback.print_exc()
            if attempt < max_retries - 1:
                time.sleep(3)
            continue
    
    return f"[错误: 推理失败，已重试{max_retries}次]"

print("✅ Qwen3 智能对比测试函数已加载（纯流式，无降级）")
print("📊 支持模型: qwen3:8b, qwen3:14b")
#TODO 用l4的时候解除注释
# print("📊 L4 GPU: qwen3:32b 也可用")
print("⚡ 参数配置: num_ctx=8192, num_predict=4096, repeat_penalty=1.07")
print("⚡ 无timeout、无num_ctx、无num_predict限制")
print("🌊 流式显示：实时打印每个token，像ChatGPT一样")
print("💡 调试: debug=True, 显示思考: show_thinking=True")

In [ ]:
# ========== Qwen3 智能对比测试实验（流式显示 + 无截断）==========

def experiment_8b_vs_14b_intelligence(debug=False, models=None, 
                                      enable_thinking=False, show_thinking=False):
    """
    对比不同规模 Qwen3 模型的智能程度差异
    
    ⚡ 优化: 按模型循环而非按问题循环，减少模型切换次数，提升显存效率
    🔧 修复: 禁用thinking模式，避免thinking占用所有token导致content为空
    🌊 流式显示: 实时打印每个token，完整显示所有内容
    
    参数:
        debug: 是否开启调试模式
        models: 要测试的模型列表，默认为 ["qwen3:8b", "qwen3:14b"]
                #TODO 用l4的时候解除注释: 可以添加 "qwen3:32b"
        enable_thinking: 是否启用Qwen3的思考模式（默认False，推荐关闭）
        show_thinking: 是否显示思考过程（仅当enable_thinking=True时有效）
    """
    
    if models is None:
        models = ["qwen3:8b", "qwen3:14b"]
        #TODO 用l4的时候解除注释
        # models = ["qwen3:8b", "qwen3:14b", "qwen3:32b"]
    
    print("=" * 80)
    print("🎯 Qwen3 智能对比测试（流式显示 + 无截断）")
    print("=" * 80)
    print(f"📊 测试模型: {', '.join(models)}")
    print(f"🔧 调试模式: {'开启' if debug else '关闭'}")
    print(f"💭 思考模式: {'开启' if enable_thinking else '关闭（推荐）'}")
    print(f"⚡ 优化策略: 每个模型完整运行所有问题后再切换（减少显存搬运）")
    print(f"🌊 显示模式: 实时流式显示，完整内容无截断")
    print("=" * 80)
    print()
    
    # 精心设计的测试问题集
    test_questions = [
        {
            "id": 1,
            "category": "逻辑推理",
            "question": "如果所有的猫都会爬树，而小花是一只猫，那么小花会爬树吗？请解释你的推理过程。",
            "metrics": ["logic"]
        },
        {
            "id": 2,
            "category": "数学推理",
            "question": "一个水池有一个进水管和一个出水管。进水管单独开需要6小时注满水池，出水管单独开需要8小时排空满池。如果两个水管同时打开，需要多长时间才能注满空池？",
            "metrics": ["math", "logic"]
        },
        {
            "id": 3,
            "category": "知识理解",
            "question": "请解释量子纠缠现象，并举一个实际应用的例子。",
            "metrics": ["knowledge", "depth"]
        },
        {
            "id": 4,
            "category": "创意写作",
            "question": "请以'时间旅行者意外改变了历史'为题，写一段100字左右的科幻短文。",
            "metrics": ["creativity"]
        },
        {
            "id": 5,
            "category": "代码能力",
            "question": "请用Python写一个函数，找出列表中出现次数最多的元素，并返回该元素及其出现次数。",
            "metrics": ["code"]
        },
        {
            "id": 6,
            "category": "伦理思考",
            "question": "如果AI技术能够完美预测人类的犯罪行为，我们应该在犯罪发生前就逮捕嫌疑人吗？为什么？",
            "metrics": ["ethics", "depth"]
        },
        {
            "id": 7,
            "category": "抽象思维",
            "question": "请解释'自由意志'这个概念，并讨论它是否可能存在。",
            "metrics": ["philosophy", "depth"]
        },
        {
            "id": 8,
            "category": "复杂推理",
            "question": "有三个人站在桥上：一个总是说真话，一个总是说谎，一个有时说真话有时说谎。你只能问其中一个人一个问题，如何确定哪个是说真话的人？",
            "metrics": ["logic", "complexity"]
        }
    ]
    
    # 存储所有模型的所有问题回答
    # 结构: {model: {question_id: response}}
    all_responses = {model: {} for model in models}
    
    # ===== 核心改变：外层循环模型，内层循环问题 =====
    # 这样可以让每个模型保持加载状态，减少显存切换
    for model_idx, model in enumerate(models):
        model_name = model.replace("qwen3:", "Qwen3 ").replace("b", "B")
        
        print("\n" + "="*80)
        print(f"🚀 开始测试 {model_name} ({model_idx+1}/{len(models)})")
        print("="*80)
        print(f"💡 此模型将连续回答 {len(test_questions)} 个问题")
        print()
        
        # 遍历所有问题
        for q in test_questions:
            print(f"\n{'─'*80}")
            print(f"📝 [{model_name}] 问题 {q['id']}/{len(test_questions)}: {q['category']}")
            print(f"{'─'*80}")
            print(f"❓ {q['question']}")
            print()
            
            messages = [{"role": "user", "content": q['question']}]
            
            try:
                response = ollama_chat_qwen(
                    messages=messages,
                    model=model,
                    temperature=0.7,
                    debug=debug,
                    enable_thinking=enable_thinking,
                    show_thinking=show_thinking
                )
                
                all_responses[model][q['id']] = {
                    'question': q,
                    'response': response,
                    'success': not response.startswith("[错误")
                }
                
                # 注意：流式模式下，回答已经在 ollama_chat_qwen 中实时打印了
                # 这里只显示统计信息
                print(f"📏 完整长度: {len(response)} 字符")
                
            except Exception as e:
                error_msg = f"[错误: {str(e)}]"
                all_responses[model][q['id']] = {
                    'question': q,
                    'response': error_msg,
                    'success': False
                }
                print(f"❌ {error_msg}")
            
            print()
            time.sleep(2)  # 问题间短暂延迟
        
        print(f"\n✅ {model_name} 完成所有 {len(test_questions)} 个问题")
        print(f"{'='*80}")
        
        # 模型间延迟，给显存切换留出时间
        if model_idx < len(models) - 1:
            print(f"\n⏳ 准备切换到下一个模型，等待5秒...")
            time.sleep(5)
    
    # 打印对比分析和总结
    print("\n\n" + "="*80)
    print("📊 跨模型对比分析")
    print("="*80)
    print_comparison_analysis(all_responses, models, test_questions)
    
    print("\n" + "="*80)
    print("📈 实验总结")
    print("="*80)
    print_summary(all_responses, models)
    
    return all_responses


def print_comparison_analysis(all_responses, models, test_questions):
    """打印每个问题的跨模型对比分析"""
    
    for q in test_questions:
        print(f"\n{'─'*80}")
        print(f"📝 问题 {q['id']}: {q['category']}")
        print(f"{'─'*80}")
        
        # 收集该问题的所有模型回答
        question_responses = {}
        for model in models:
            if q['id'] in all_responses[model]:
                question_responses[model] = all_responses[model][q['id']]['response']
        
        # 基础统计
        print("📊 回答长度对比:")
        for model in models:
            if model in question_responses:
                response = question_responses[model]
                model_name = model.replace("qwen3:", "").upper()
                
                if response.startswith("[错误"):
                    print(f"   {model_name}: ❌ 失败")
                else:
                    print(f"   {model_name}: {len(response)} 字符")
        
        # 特定类别分析
        analyze_question_category(q, question_responses, models)


def analyze_question_category(question, responses, models):
    """根据问题类别进行针对性分析"""
    
    category = question['category']
    
    if "逻辑" in category or "推理" in category:
        print("\n🧠 逻辑性指标:")
        logic_words = ["因为", "所以", "如果", "那么", "推理", "结论", "前提", "必然"]
        for model in models:
            if model in responses:
                response = responses[model]
                if not response.startswith("[错误"):
                    count = sum(1 for word in logic_words if word in response)
                    model_name = model.replace("qwen3:", "").upper()
                    print(f"   {model_name}: {count} 个逻辑词")
    
    elif "数学" in category:
        print("\n🔢 数学元素指标:")
        math_indicators = ["计算", "公式", "方程", "小时", "÷", "×", "+", "-", "=", "速度", "效率"]
        for model in models:
            if model in responses:
                response = responses[model]
                if not response.startswith("[错误"):
                    count = sum(1 for word in math_indicators if word in response)
                    model_name = model.replace("qwen3:", "").upper()
                    print(f"   {model_name}: {count} 个数学元素")
    
    elif "知识" in category:
        print("\n📚 知识深度指标:")
        knowledge_words = ["原理", "机制", "应用", "技术", "现象", "理论", "量子", "纠缠"]
        for model in models:
            if model in responses:
                response = responses[model]
                if not response.startswith("[错误"):
                    count = sum(1 for word in knowledge_words if word in response)
                    model_name = model.replace("qwen3:", "").upper()
                    print(f"   {model_name}: {count} 个专业术语")
    
    elif "代码" in category:
        print("\n💻 代码质量指标:")
        code_elements = ["def", "return", "if", "for", "while", "try", "except", "import", "dict"]
        for model in models:
            if model in responses:
                response = responses[model]
                if not response.startswith("[错误"):
                    count = sum(1 for elem in code_elements if elem in response)
                    has_code = "```" in response or "def " in response
                    model_name = model.replace("qwen3:", "").upper()
                    print(f"   {model_name}: {count} 个代码元素, 代码块: {'有' if has_code else '无'}")
    
    elif "创意" in category:
        print("\n✨ 创意性指标:")
        creative_words = ["突然", "意外", "发现", "惊讶", "奇妙", "神秘", "时间", "穿越"]
        for model in models:
            if model in responses:
                response = responses[model]
                if not response.startswith("[错误"):
                    count = sum(1 for word in creative_words if word in response)
                    model_name = model.replace("qwen3:", "").upper()
                    print(f"   {model_name}: {count} 个创意词汇")
    
    elif "伦理" in category or "抽象" in category:
        print("\n🤔 思维深度指标:")
        depth_words = ["但是", "然而", "考虑", "平衡", "道德", "伦理", "哲学", "观点", "角度"]
        for model in models:
            if model in responses:
                response = responses[model]
                if not response.startswith("[错误"):
                    count = sum(1 for word in depth_words if word in response)
                    model_name = model.replace("qwen3:", "").upper()
                    print(f"   {model_name}: {count} 个深度思考词")


def print_summary(all_responses, models):
    """打印实验总结统计"""
    
    print("\n🎯 整体表现对比:")
    
    for model in models:
        model_name = model.replace("qwen3:", "Qwen3 ").replace("b", "B")
        
        success_count = 0
        total_length = 0
        
        for q_id, data in all_responses[model].items():
            if data['success']:
                success_count += 1
                total_length += len(data['response'])
        
        total_questions = len(all_responses[model])
        avg_length = total_length // success_count if success_count > 0 else 0
        success_rate = (success_count / total_questions) * 100 if total_questions > 0 else 0
        
        print(f"\n{model_name}:")
        print(f"  ✓ 成功率: {success_count}/{total_questions} ({success_rate:.1f}%)")
        print(f"  📏 平均回答长度: {avg_length} 字符")
        
        if success_count > 0:
            # 找出最长和最短的回答
            lengths = [len(data['response']) for data in all_responses[model].values() if data['success']]
            print(f"  📊 回答长度范围: {min(lengths)} - {max(lengths)} 字符")
    
    print("\n💡 优化效果:")
    print(f"  ⚡ 模型切换次数: {len(models) - 1} 次（相比问题循环减少了 {(len(all_responses[models[0]]) - 1) * (len(models) - 1)} 次）")
    
    print("\n" + "="*80)
    print("✅ 实验完成！")
    print("="*80)


# 默认运行实验
print("🚀 准备运行实验...")
print("💡 提示: 使用 experiment_8b_vs_14b_intelligence(debug=True) 查看详细调试信息")
print("💡 思考模式: experiment_8b_vs_14b_intelligence(enable_thinking=True, show_thinking=True)")
#TODO 用l4的时候解除注释: 添加32B模型
# print("💡 L4 GPU: 使用 experiment_8b_vs_14b_intelligence(models=['qwen3:8b', 'qwen3:14b', 'qwen3:32b'])")
print()

# 执行实验
experiment_8b_vs_14b_intelligence()